<a href="https://colab.research.google.com/github/danakurniawan-backup/algorithms_search_maze/blob/main/maze.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import time
import resource
from queue import Queue
import heapq

import argparse

if __name__ == "__main__":
	parser = argparse.ArgumentParser(description='Robot Path Planning')
	parser.add_argument('-bfs', action="store_true", default=False , help="Run BFS on the map")
	parser.add_argument('-dfs', action="store_true", default=False, help= "Run DFS on the map")
	parser.add_argument('-astar', action="store_true", default=False, help="Run A* on the map")
	parser.add_argument('-ida', action="store_true", default=False, help="Run Iterative Deepening A* on the map")
	parser.add_argument('-all', action="store_true", default=False, help="Run all the 4 algorithms")
	parser.add_argument('-m', action="store", help="Map filename")

	results = parser.parse_args(['-bfs', '-dfs', '-astar', '-ida', '-m', 'arena1.txt'])
 #['-bfs', '-dfs', '-astar', '-ida', '-m', 'arena1.txt']

	if results.m=="" or not(results.all or results.astar or results.bfs or results.dfs or results.ida):
		print("Check the parameters : >> python hw1_UNI.py -h")
		exit()

	if results.all:
		results.bfs = results.dfs = results.astar = results.ida = True

	# Reading of map given and all other initializations
	try:
		with open(results.m) as f:
			arena = f.read()
			arena = arena.split("\n")
	except:
		print("Error in reading the arena file.")
		exit()

	# Internal representation
	print(arena)

	print("The arena of size "+ str(len(arena)) + "x" + str(len(arena[0])))
	print("\n".join(arena))

class MazeState:
	'''
	This class is an abstraction to store a maze state, which contains the following:
	- Maze configuration (arena)
	- Current Position (position in the the maze that the current state represents)
	- Parent (the state from which the current state came from)
	- Action (the action taken in the parent state, direction moved, which lead to the creation of the current state)
	- Cost (Cost  of the path taken from the start to the current state)
	- Children (a child of the current state is generated by moving in a direction)
	'''

	def get_start_index(self):
		'''
		Returns the start index of the maze based on the given arena
		returns (-1, -1) if no start index found
		'''
		for row in range(len(self.arena)):
			for col in range(len(self.arena[row])):
				if self.arena[row][col] == 's':
					return(row, col)
		return (-1, -1)

	def get_goal_index(self):
		'''
		Returns the goal index of the maze based on the given arena
		returns (-1, -1) if no goal index found
		'''
		for row in range(len(self.arena)):
			for col in range(len(self.arena[row])):
				if self.arena[row][col] == 'g':
					return (row, col)
		return (-1, -1)

	def __init__(self, arena, parent=None, action='Start', cost=0, current_position=(-1,-1)):

		self.arena = arena
		self.parent = parent
		self.action = action
		self.cost = cost
		self.children = []

		self.start = self.get_start_index()
		self.goal = self.get_goal_index()

		if(current_position[0] == -1):
			self.current_position = self.start
		else:
			self.current_position = current_position

		def display(self):
			print("\n".join(self.arena))

	def move_up(self):
		'''
	This function checks if up is a valid move from the given state.
	If up is a valid move, returns a child in which the player has moved up
	Else returns None.
	'''
		current_row, current_col = self.current_position

		#making sure move is within bounds
		if current_row == 0:
			return None
		#move up = subtract row by 1, so move up
		new_row, new_col = current_row - 1, current_col
		#if move up is into an obstacle
		if self.arena[new_row][new_col] == 'o':
			return None

		#returns a child in the form of a MazeState object
		new_state = MazeState(
				self.arena,
				parent = self,
				action = 'UP',
				cost = self.cost + 1,
				current_position = (new_row, new_col)
		)
		return new_state

	def move_down(self):
		'''
		This function checks if down is a valid move from the given state.
		If down is a valid move, returns a child in which the player has moved down.
		Else returns None.
		'''
		current_row, current_col = self.current_position

		#making sure move is within bounds
		if current_row == len(self.arena) - 1:
			return None
		#move down = add to row by 1, so move down
		new_row, new_col = current_row + 1, current_col
		#if move up is into an obstacle
		if self.arena[new_row][new_col] == 'o':
			return None

		#returns a child in the form of a MazeState object
		new_state = MazeState(
				 self.arena,
				 parent = self,
				 action = 'Down',
				 cost = self.cost + 1,
				 current_position = (new_row, new_col)
		)
		return new_state

	def move_left(self):
		'''
		This function checks if left is a valid move from the given state.
		If left is a valid move, returns a child in which the player has moved left.
		Else returns None.
		'''

		current_row, current_col = self.current_position

		#check if current position is already most left and cannot go further
		if current_col == 0:
			return None
		#move left = subtract column by 1, moves left
		new_row, new_col = current_row, current_col - 1

		#if new position is obstacle
		if self.arena[new_row][new_col] == 'o':
			return None

		#returns a child in the form of a MazeState object
		new_state = MazeState(
				 self.arena,
				 parent = self,
				 action = 'Left',
				 cost = self.cost + 1,
				 current_position = (new_row, new_col)
		)
		return new_state

	def move_right(self):
		'''
		This function checks if left is a valid move from the given state.
		If left is a valid move, returns a child in which the player has moved left.
		Else returns None.
		'''

		current_row, current_col = self.current_position

		#check if we are already at right most position
		if current_col == len(self.arena[0]) - 1:
			return None

		#if valid move, move right by 1 column
		new_row, new_col = current_row, current_col + 1

		#if new position is obstacle
		if self.arena[new_row][new_col] == 'o':
			return None

		#returns a child in the form of a MazeState object
		new_state = MazeState(
				 self.arena,
				 parent = self,
				 action = 'Right',
				 cost = self.cost + 1,
				 current_position = (new_row, new_col)
		)
		return new_state
	def expand(self):
		"""
		Generate the child nodes of this node
		"""

		if(len(self.children) != 0):
			return self.children

		# Do not change the order in this function, since the grading script assumes this order of expansion when checking
		children = [self.move_up(), self.move_right(), self.move_down(), self.move_left()]

		self.children = [state for state in children if state is not None]
		return self.children

	def __hash__(self):
		'''
		Maze states hashed based on cost.
		This function may be modified if required.
		'''

		return self.cost

	def __eq__(self, other):

		m1 = self.arena
		m2 = other.arena

		if(len(m1) != len(m2)):
			return False

		for i in range(0, len(m1)):
			if(not (m1[i] == m2[i])):
				return False

		return self.current_position == other.current_position

	'''		Maze states are defined as equal if they have the same dimensions and the same current position.
			This function may be modified if required.
	def __lt__(self,other):
		return self.current_position < other.current_position

def get_max_ram_usage():
	dfs_start_ram = resource.getrusage(resource.RUSAGE_SELF).ru_maxrss
	dfs_ram = (resource.getrusage(resource.RUSAGE_SELF).ru_maxrss - dfs_start_ram)/(2**10)
	#gets RAM usage in kilobytes
	return dfs_ram


'''
This function runs Breadth First Search on the input arena (which is a list of str)
Returns a ([], int) tuple where the [] represents the solved arena as a list of str and the int represents the cost of the solution
'''
def bfs(arena):

	start_time = time.time()
	bfs_cost = 0
	bfs_ram = 0
	bfs_max_nodes_stored = 0
	bfs_max_search_depth = 0
	bfs_nodes_expanded = 0
	bfs_arena = []
	bfs_time = 0

	#initialize start
	starting_state = MazeState(arena)

	if starting_state.start == (-1, -1) or starting_state.goal == (-1, -1):
		return([], 0, 0, 0, 0, 0, 0)

	#implement an explicit queue as seen in lecture, start by removing a node from frontier
	frontier = Queue()
	frontier.put(starting_state)
	explored = set() #initialize empty set of what has already been explored
	path_to_goal = arena.copy()

	while not frontier.empty():
		current_state = frontier.get()

		# print(f"\n\nIteration {iter}\nCurrent_state's current_position: {current_state.current_position}")
		# print(f"Current Cost: {current_state.cost}")

		#check if node matches the goal state
		if current_state.current_position == current_state.goal:
			#print(f"Count when we get to goal: {iter}")
			path_to_goal = arena.copy()
			bfs_cost = current_state.cost
			current = current_state
			#return (bfs_arena, bfs_cost, bfs_nodes_expanded, bfs_max_nodes_stored, bfs_max_search_depth, bfs_time, bfs_ram)
			while current.parent:
				if current.current_position == current_state.goal:
					current = current.parent
					continue
					#may be running it for more than one step, need to trace back and make sure it shouldn't run everything else, maybe queueing nodes from goal state accidentally
					#try having it manually print out what's in the frontier, what's being explored
					#rather than continue, put return statement
				row, col = current.current_position
				path_to_goal[row] = path_to_goal[row][:col] + '*' + path_to_goal[row][col + 1:]
				current = current.parent
			bfs_arena = path_to_goal

			end_time = time.time()
			bfs_time = end_time - start_time
			break

		#add to explored set
		# explored.add(current_state)
		explored.add(current_state.current_position)
		bfs_nodes_expanded += 1
		#return (bfs_arena, bfs_cost, bfs_nodes_expanded, bfs_max_nodes_stored, bfs_max_search_depth, bfs_time, bfs_ram)

		#expand the node, generate all immediate successors and add them to the frontier
		#expand nodes in order Up, Right, Down, Left
		for move in [current_state.move_up, current_state.move_right, current_state.move_down, current_state.move_left]:
			child = move()

			#check if child exists, and not already explored, and not stored in frontier, add
			if child and child.current_position not in explored and child not in frontier.queue:
				frontier.put(child)
				#maybe max is too large here  - may still be expanding once algorithm has terminated
				bfs_max_nodes_stored = max(bfs_max_nodes_stored, frontier.qsize() + len(explored))
				bfs_max_search_depth = max(bfs_max_search_depth, child.cost)
				bfs_ram = max(bfs_ram, get_max_ram_usage())

		#bfs_max_search_depth = max(bfs_max_search_depth, current_state.cost)
	#print("count:", iter)
	return (bfs_arena, bfs_cost, bfs_nodes_expanded, bfs_max_nodes_stored, bfs_max_search_depth, bfs_time, bfs_ram)
'''
This function runs Depth First Search on the input arena (which is a list of str)
Returns a ([], int) tuple where the [] represents the solved arena as a list of str and the int represents the cost of the solution
#was told we could ignore and return the 7 results
'''

def dfs(arena):

	start_time = time.time()
	dfs_cost = 0
	dfs_ram = 0
	dfs_max_nodes_stored = 0
	dfs_max_search_depth = 0
	dfs_nodes_expanded = 0
	dfs_arena = []
	dfs_time = 0

	#initialize start
	starting_state = MazeState(arena)
	#print("Starting state:", starting_state.arena, starting_state.start, starting_state.goal, starting_state.current_position)
	if starting_state.start == (-1, -1) or starting_state.goal == (-1, -1):
		return ([], -1, 0, 0, 0, 0, 0)

	#implement a stack (LIFO - Last in First Out) as seen in lecture
	frontier = [starting_state]
	#print("Frontier:", frontier)
	explored = set() #initialize empty set of what has already been explored

	#print(f"Is starting state in frontier? (it should be): {starting_state in frontier}")

	#print(starting_state)
	#print(len(frontier))
	iter = 0
	max_iter = 20
	while frontier:
		current_state = frontier.pop()
		#print(f"\n\nIteration {iter}\nCurrent_state's current_position: {current_state.current_position}")
		#print(f"goal: {current_state.goal}")

		#check if we have reached goal state
		if current_state.current_position == current_state.goal:
			#print("Current position:", current_state.current_position)
			#print("Goal position:", current_state.goal)
			path_to_goal = arena.copy()
			dfs_cost = current_state.cost
			current = current_state
			while current.parent:
				#trying to see if this is redundant or helps like in BFS above
				if current.current_position == current_state.goal:
					current = current.parent
					continue
				row, col = current.current_position
				path_to_goal[row] = path_to_goal[row][:col] + '*' + path_to_goal[row][col + 1:]
				current = current.parent
			dfs_arena = path_to_goal
			break

		explored.add(current_state.current_position) # Used to be .add(current_state)
		dfs_nodes_expanded += 1
		#print(f"frontier length: {len(frontier)}")
		#print(f"explored length: {len(explored)}")
		#for item in frontier:
			#print(f"Frontier item's position: {item.current_position}")
		#for item in explored:
		#	print(f"Explored item's position: {item}")

		#follow order: up, right, down, left
		for move in [current_state.move_left, current_state.move_down, current_state.move_right, current_state.move_up]:
			child = move()
	 		# This check is somehow incorrect, it adds children that are already in explored
			if child and child.current_position not in explored and child not in frontier:
				#print(f"Is child in fronter? (It shouldn't be): {child in frontier}, Position ({child.current_position})")
				#print(f"Is child in explored? (It shouldn't be): {child in explored}")
				frontier.append(child)
				#print(f"Added child to frontier with position {child.current_position}")
				dfs_max_nodes_stored = max(dfs_max_nodes_stored, len(frontier) + len(explored))
				dfs_max_search_depth = max(dfs_max_search_depth, child.cost)
				dfs_ram = max(dfs_ram, get_max_ram_usage())
		#print(len(frontier))

	end_time = time.time()
	dfs_time = end_time - start_time
	return (dfs_arena, dfs_cost, dfs_nodes_expanded, dfs_max_nodes_stored, dfs_max_search_depth, dfs_time, dfs_ram) # Replace with return values
	#=================================#
	#*#*#*# Your code ends here #*#*#*#
	#=================================#

'''
This function runs A* Search on the input arena (which is a list of str)
Returns a ([], int) tuple where the [] represents the solved arena as a list of str and the int represents the cost of the solution
'''
def astar(arena):
	#================================================#
	#*#*#*# TODO: Write your A* algorithm here #*#*#*#
	#================================================#

	#see if the same value for child and by time added, sort by it, should get same value.
	start_time = time.time()
	astar_cost = 0
	astar_ram = 0
	astar_max_nodes_stored = 0
	astar_max_search_depth = 0
	astar_nodes_expanded = 0
	astar_arena = []
	astar_time = 0

	starting_state = MazeState(arena)
	if starting_state.start == (-1, -1) or starting_state.goal == (-1, -1):
		return ([], 1)

	#helper function for calculating heuristic, calculates manhattan distance
	def heuristic(state):
		return abs(state.current_position[0] - state.goal[0]) + abs(state.current_position[1] - state.goal[1])
	#print(f"current position: {starting_state}")

	frontier = [(heuristic(starting_state), starting_state)]
	heapq.heapify(frontier)
	explored = set()
	iter = 0

	while frontier:
		cost, current_state = heapq.heappop(frontier)

		if current_state.current_position in explored and cost >= current_state.cost:
			continue
		astar_nodes_expanded += 1

		#check goal state
		if current_state.current_position == current_state.goal:
			astar_arena = arena.copy()
			astar_cost = current_state.cost
			current = current_state
			while current.parent:
				if current.current_position == current_state.goal:
					current = current.parent
					continue
				row, col = current.current_position
				#mark path with *
				astar_arena[row] = astar_arena[row][:col] + '*' + astar_arena[row][col + 1:]
				current = current.parent
			end_time = time.time()
			astar_time = end_time - start_time
			return (astar_arena, astar_cost, astar_nodes_expanded, astar_max_nodes_stored, astar_max_search_depth, astar_time, astar_ram)

		explored.add(current_state.current_position)

		for move in [current_state.move_up, current_state.move_right, current_state.move_down, current_state.move_left]:
			child = move()
			if child and child.current_position not in explored:
				#cost of adding child is + heuristic
				child_cost = child.cost + heuristic(child)
				#see if child node is already in the frontier set (tuple containing the cost and MazeState object)
				if child not in [state for _, state in frontier]:
					heapq.heappush(frontier, (child_cost, child))
					#max nodes stored
					astar_max_nodes_stored = max(astar_max_nodes_stored, len(frontier) + len(explored))
					astar_max_search_depth = max(astar_max_search_depth, child.cost)
					astar_ram = max(astar_ram, get_max_ram_usage())
				else:
					for i, (_, state) in enumerate(frontier):
						if state == child and child_cost < frontier[i][0]:
							frontier[i] = (child_cost, child)
							heapq.heapify(frontier)
	return (astar_arena, astar_cost, astar_nodes_expanded, astar_max_nodes_stored, astar_max_search_depth, astar_time, astar_ram)
	#=================================#
	#*#*#*# Your code ends here #*#*#*#
	#=================================#

#making global, I got confused with all the return statements
ida_cost = 0
ida_nodes_expanded = 0
ida_max_nodes_stored = 1  # We have to store at least the starting state
ida_max_search_depth = 0
ida_ram = 0
ida_time = 0

def ida(arena):

	#=================================================#
	#*#*#*# TODO: Write your IDA algorithm here #*#*#*#
	#=================================================#

	start_time = time.time()

	starting_state = MazeState(arena)
	if starting_state.start == (-1, -1) or starting_state.goal == (-1, -1):
		return ([], -1)

	bound = heuristic(starting_state)  # How deep we will initially search
	path = [starting_state]  # Has to function like a stack, stores the nodes we have searched through

	# This is the starting depth, which might change as we go deeper
	global ida_max_search_depth
	ida_max_search_depth = max(ida_max_search_depth, bound)

	iter = 0
	# max_iter = 20
	while True:
		output = _search(path, 0, bound)
		if output == 'found':
			# print(f"Found it!")
			path_node = path[-1]
			ida_arena = get_path_arena(path_node, arena)

			ida_cost = bound  # We deepen the bound (by the minimum cost of moving to a child node) until we find the solution
			end_time = time.time()
			ida_time = end_time - start_time
			return ida_arena, ida_cost, ida_nodes_expanded, ida_max_nodes_stored, ida_max_search_depth, ida_time, ida_ram
		if output == float('inf'):
			found = "this is our value"
			return found
		else:
			bound = output

		iter += 1

def heuristic(state):
	# The l1 norm = manhattan distance vertical plus horizontal
	return abs(state.current_position[0] - state.goal[0]) + abs(state.current_position[1] - state.goal[1])

def is_goal(state):
	# Checks whether the given maze state is at the goal
	return state.current_position == state.goal

def get_successors(state):
	# Returns the children sorted by g+h, with cost g (source->state) and heuristic h(state->goal)
	successors = []
	for move in [state.move_up, state.move_right, state.move_down, state.move_left]:
		child = move()
		# Check if the child exists (move will return None if it doesn't), then calculate g+h and sort
		if child:
			gh = child.cost + heuristic(child)
			successors.append((gh, child))
	return successors

def get_path_arena(end_state, arena):
	# For when you've found the solution, work back from the end state and print asterisks at the locations
	ida_path = arena.copy()
	while end_state.parent:
		if is_goal(end_state):
			# Skip the first location to not override the goal location
			end_state = end_state.parent
		else:
			row, col = end_state.current_position
			#mark path with *
			ida_path[row] = ida_path[row][:col] + '*' + ida_path[row][col + 1:]  # Because they look like dispenses
			end_state = end_state.parent
	return ida_path

def _search(path, g, bound):  # g is the cost to get from source to the current node (blame wikipedia for the poor naming I only figured it out at the end)
	node = path[-1]  # The last element of the path is the node we're currently working on (stack structure)
	heur_cost = g + heuristic(node)  # Estimate of the total cost based on the current path

	if heur_cost > bound:
		# We've exceeded the depth of search, and need to update and go deeper
		global ida_max_search_depth  # Access global variable
		ida_max_search_depth = max(ida_max_search_depth, heur_cost)
		return heur_cost
	if is_goal(node):
		return 'found'

	min = float('inf')

	# Otherwise we need to search through the children of this node
	for (_, successor) in get_successors(node):
		global ida_nodes_expanded
		ida_nodes_expanded += 1
		# If the successor is not in path (prevents us from going backwards)
		if successor.current_position not in [past_nodes.current_position for past_nodes in path]:
			path.append(successor)
			global ida_max_nodes_stored  # Access these global variable
			global ida_ram
			ida_max_nodes_stored = max(ida_max_nodes_stored, len(path))  # Update this statistic if we've exceeded the previous record
			ida_ram = max(ida_ram, get_max_ram_usage())

			# Named this so we know it's at the inner level
			output_inner = _search(path, g + 1, bound)
			if output_inner == 'found':
				return 'found'
			if output_inner < min:
				min = output_inner

			path.pop()
	return min

# 	start_time = time.time()
# 	ida_cost = 0
# 	ida_nodes_expanded = 0
# 	ida_max_nodes_stored = 0
# 	ida_max_search_depth = 0
# 	ida_ram = 0
# 	ida_time = 0
# 	ida_arena = []


# 	starting_state = MazeState(arena)
# 	if starting_state.start == (-1, -1) or starting_state.goal == (-1, -1):
# 		return ([], -1)

# 	def heuristic(state):
# 		return abs(state.current_position[0] - state.goal[0]) + abs(state.current_position[1] - state.goal[1])

# 	# cost_f = state.cost + heuristic(state) # f = g + h # hopy here's where I think I need to incorporate f = g + h and not just cutoff when the limit is too high
# 	# if cost_f > limit:
# 	# 	if cost_f < min_over_limit_cost[0]:
# 	# 		min_over_limit_cost[0] = cost_f
# 	# 	cutoff_happened = True

# 	print(f"Beginning IDA Iteration")
# 	depth = 0
# 	while True:
# 		print(f"\nBeginning _depth_limited_search")
# 		result, ida_arena, ida_nodes_expanded, ida_max_nodes_stored, ida_max_search_depth = _depth_limited_search(starting_state, depth, starting_state.arena.copy(), 0, 0, current_depth=0)
# 		print(f"Returned from _depth_limited_search, {depth=}, {result=}")

# 		#check if result is not cutoff value
# 		if result != "cutoff":
# 			end_time = time.time()
# 			ida_time = end_time - start_time
# 			print(f"Found the solution, back in the main function")
# 			print(f"{result=}")
# 			print("End of results")
# 			return (ida_arena, result[1], ida_nodes_expanded, ida_max_nodes_stored, ida_max_search_depth, ida_time, ida_ram)
# 		depth += 1
# 		#update depth counter
# 		ida_max_search_depth = depth
# 		print(f"Max search depth {ida_max_search_depth}")

# #wrote a helper function used within the ida function using a recursive depth limited search function
# def _depth_limited_search(state, limit, ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth):

# 	#node = current MazeState object, returns a tuple with needed information or "cutoff" if we get to the depth
# 	ida_nodes_expanded += 1
# 	current_depth += 1
# 	# print(f"Current depth: {current_depth}")

# 	#goal test
# 	if state.current_position == state.goal:
# 		print(f"\tGot to the goal, {current_depth=}")
# 		current = state
# 		iter_parent = 0
# 		while current.parent:
# 			if current.current_position == state.goal:
# 				current = current.parent
# 				continue
# 			# print(f"Iterating on parents, iteration: {iter_parent}, position: {current.current_position}")
# 			# print(f"Iterating on parents, iteration: {iter_parent}, position: {current.current_position}, parent position: {current.parent.current_position}")
# 			# print(f"{current.parent.parent.current_position}")
# 			iter_parent += 1
# 			row, col = current.current_position
# 			ida_arena[row] = ida_arena[row][:col] + '*' + ida_arena[row][col + 1:]
# 			current = current.parent
# 		return (ida_arena, state.cost), ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth

# 	elif limit == 0:
# 		# print(f"\tReached the limit==0, {current_depth=}")
# 		return "cutoff", ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth

# 	else:
# 		cutoff_happened = False
# 		#print(f"Length of state.children: {len(state.children)}")
# 		ida_max_nodes_stored = max(ida_max_nodes_stored, len(state.children) + 1)  # TODO: Check if node.children is working properly here, it wasn't on the line below
# 		for move in [state.move_left, state.move_down, state.move_right, state.move_up]:
# 			child = move()
# 			if not child:
# 				continue
# 			# print(f"\tRecursively calling _depth_limited_search(), {current_depth=}")
# 			result, ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth = _depth_limited_search(
# 					 child, limit - 1, ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth
# 					 )
# 			if result == "cutoff":
# 				cutoff_happened = True
# 			elif result != "failed":
# 				#yay found solution
# 				return result, ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth
# 		# print(f"\tChecking if cutoff_happened")
# 		if cutoff_happened:
# 			return "cutoff", ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth
# 		else:
# 			return "failed", ida_arena, ida_nodes_expanded, ida_max_nodes_stored, current_depth
	#=================================#
	#*#*#*# Your code ends here #*#*#*#
	#=================================#

if __name__ == "__main__":
	if results.bfs:
		print("\nBFS algorithm called")
		bfs_arena, bfs_cost, bfs_nodes_expanded, bfs_max_nodes_stored, bfs_max_search_depth, bfs_time, bfs_ram = bfs(arena)
		print("\n".join(bfs_arena))
		print("BFS:")
		print("Cost: " + str(bfs_cost))
		print("Nodes Expanded: " + str(bfs_nodes_expanded))
		print("Max Nodes Stored: " + str(bfs_max_nodes_stored))
		print("Max Search Depth: " + str(bfs_max_search_depth))
		print("Time: " + str(bfs_time) + "s")
		print("RAM Usage: " + str(bfs_ram) + "kB\n")

	if results.dfs:
		print("\nDFS algorithm called")
		dfs_arena, dfs_cost, dfs_nodes_expanded, dfs_max_nodes_stored, dfs_max_search_depth, dfs_time, dfs_ram = dfs(arena)
		print("\n".join(dfs_arena))
		print("DFS:")
		print("Cost: " + str(dfs_cost))
		print("Nodes Expanded: " + str(dfs_nodes_expanded))
		print("Max Nodes Stored: " + str(dfs_max_nodes_stored))
		print("Max Search Depth: " + str(dfs_max_search_depth))
		print("Time: " + str(dfs_time) + "s")
		print("RAM Usage: " + str(dfs_ram) + "kB\n")

	if results.astar:
		print("\nA* algorithm called")
		astar_arena, astar_cost, astar_nodes_expanded, astar_max_nodes_stored, astar_max_search_depth, astar_time, astar_ram = astar(arena)
		print("\n".join(astar_arena))
		print("A*:")
		print("Cost: " + str(astar_cost))
		print("Nodes Expanded: " + str(astar_nodes_expanded))
		print("Max Nodes Stored: " + str(astar_max_nodes_stored))
		print("Max Search Depth: " + str(astar_max_search_depth))
		print("Time: " + str(astar_time) + "s")

	if results.ida:
		print("\nIterative Deepening A* algorithm called")
		ida_arena, ida_cost, ida_nodes_expanded, ida_max_nodes_stored, ida_max_search_depth, ida_time, ida_ram = ida(arena)
		print("\n".join(ida_arena))
		print("Iterative Deepening A*:")
		print("Cost: " + str(ida_cost))
		print("Nodes Expanded: " + str(ida_nodes_expanded))
		print("Max Nodes Stored: " + str(ida_max_nodes_stored))
		print("Max Search Depth: " + str(ida_max_search_depth))
		print("Time: " + str(ida_time) + "s")
		print("RAM Usage: " + str(ida_ram) + "kB\n")



['        ', '  g     ', '  ooooo ', ' o      ', '  os  o ', '        ', '        ']
The arena of size 7x8
        
  g     
  ooooo 
 o      
  os  o 
        
        

BFS algorithm called
        
**g     
* ooooo 
*o      
**os  o 
 ***    
        
BFS:
Cost: 10
Nodes Expanded: 43
Max Nodes Stored: 47
Max Search Depth: 11
Time: 0.002045869827270508s
RAM Usage: 0kB


DFS algorithm called
     ***
  g*** *
  ooooo*
 o *****
  os  o 
        
        
DFS:
Cost: 14
Nodes Expanded: 40
Max Nodes Stored: 48
Max Search Depth: 32
Time: 0.0018758773803710938s
RAM Usage: 0kB


A* algorithm called
        
**g     
* ooooo 
*o      
**os  o 
 ***    
        
A*:
Cost: 10
Nodes Expanded: 21
Max Nodes Stored: 29
Max Search Depth: 10
Time: 0.001088857650756836s

Iterative Deepening A* algorithm called
        
**g     
* ooooo 
*o      
**os  o 
 ***    
        
Iterative Deepening A*:
Cost: 10
Nodes Expanded: 435
Max Nodes Stored: 11
Max Search Depth: 12
Time: 0.019816875457763672s
RAM Usag